A *degree n pillowcase-tiled surface* is represented by a tuple of four elements $(a,b,c,d) \in \text{Sym}(n)^4$
such that $abcd = 1$. A PTS is *1-cylinder* if every element of its braid group orbit has the property that $ab$ is an $n$-cycle. Our goal is to find non-cyclic 1-cylinder PTS, in other words, tuples such that 
* for every $g \in B_4$, if $g\cdot(a,b,c,d) = (a',b',c',d')$ then $a'b'$ is an $n$-cycle, and 
* $\langle a,b,c,d\rangle$ is a non-cyclic subgroup of $\text{Sym}(n)$.  

We're particularly interested if there are examples where the permutations $a,b,c,d$ do not include the identity permutation.

We use the class `Constellation` from the `sage::combinat` library. A *constellation* is a tuple of permutations in $\text{Sym(n)}$ whose product is the identity. The *length* of a constellation is the number of elements in the tuple,
and the *degree* is the $n$ of the symmetric group the permutations come from.
The `constellation` library by default only returns connected constellations, i.e. 
constellations whose permutations have a connected orbit when acting on 
$\{1,\dots,n\}$. In order to be a 1-cylinder surface, a PTS must be represented by a connected 
constellation, so this is ok.

In [ ]:
from sage.all import SymmetricGroup
from sage.combinat import constellation

n = 5
Sn = SymmetricGroup(n)
all_pts = Constellations(4,n) # only returns connected constellations
horizontal_one_cyl_set = set()
for tuple in all_pts:
    # tuple.g(i) is the permutation in the ith entry of the tuple
    if (tuple.g(0)*tuple.g(1)).cycle_type() == [n]:
        horizontal_one_cyl_set.add(tuple)
print(f'Total number of degree {n} PTS: {len(all_pts)}')
print(f'Total number of degree {n} PTS with a horizontal cyl: {len(horizontal_one_cyl_set)}')

Total number of degree 5 PTS: 1647384
Total number of degree 5 PTS with a horizontal cyl: 345600


In [274]:
def check_horizontal_direction(tuple,n):
    if (tuple.g(0)*tuple.g(1)).cycle_type() == [n]:
        return True 
    else:
        return False
    
def check_one_cyl_surface(tuple,n):
    G = tuple.braid_group_orbit()
    for t in G.vertices(): # t not nec = tuple, but will be isom. constellations
        if not check_horizontal_direction(t,n):
            return False
    return True

def check_for_identity(tuple,n):
    for i in range(0,4):
        if tuple.g(i).cycle_type() == ([1]*n):
            return True 
    return False

def check_cyclic(tuple,n):
    G = SymmetricGroup(n).subgroup([tuple.g(0), tuple.g(1), tuple.g(2), tuple.g(3)])
    return G.is_cyclic()

def display_info(tuple,n):
    print(f'Information about {tuple}\n')
    if check_one_cyl_surface(tuple,n):
        print(f'one-cylinder: yes\n')
    else:
        print(f'one-cylinder: no\n')
    
    print(f'Tuple profile: {tuple.profile()}\n')

def print_PTS(tuple, n, verbose=False):
    if not verbose:
        print(f'({tuple.g(0)},  {tuple.g(1)},  {tuple.g(2)},  {tuple.g(3)})')

    if verbose:
        print('PTS defined by:')
        print(f' a: {tuple.g(0)}')
        print(f' b: {tuple.g(1)}')
        print(f' c: {tuple.g(2)}')
        print(f' d: {tuple.g(3)}\n')
    
        print(f'cycle type: {tuple.profile()}')
        print(f'is cyclic: {check_cyclic(tuple,n)}')
        print(f'ab = {tuple.g(0)*tuple.g(1)}')
        print(f'is 1-cylinder: {check_one_cyl_surface(tuple,n)}')
        print(f'Braid group orbit:')
        for t in tuple.braid_group_orbit().vertices():
            print_PTS(t,n)
            print(f'a\'b\'={t.g(0)*t.g(1)}')
        print(f'\n')



In [ ]:
test_set = horizontal_one_cyl_set.copy()
count = 0
interesting = set() # all 1-cylinder surfaces
skipped = 0 # double check we're looking at all of them

for tuple in horizontal_one_cyl_set:
    if check_one_cyl_surface(tuple,n):
        interesting.add(tuple)
    else:
        skipped += 1
    # can also try to be clever about only checking orbits once, but turns out 
    # to run in about the same amount of time
    # try:
    #     test_set.remove(tuple)
    # except:
    #     continue
    # G = tuple.braid_group_orbit()
    # iso_class_already_found = False
    # for t in G.vertices(): # this is working with isomorphism classes 
    #     try:
    #         test_set.remove(t)
    #     except: # means we've already removed the iso class from the set
    #         iso_class_already_found = True
    # if not iso_class_already_found:
    #     for t in G.vertices():
    #         if not check_horizontal_direction(t,n):
    #             break
    #     interesting.add(G.vertices()[0])
        
    
print(len(interesting))
print(skipped)
print(len(interesting)+skipped)

69120
276480
345600


In [ ]:
print(f'Total number of degree {n} one-cylinder PTS: {len(interesting)}')

Total number of degree 5 one-cylinder PTS: 69120


In [232]:
cyclic_one_cyl = set()
non_cyclic_one_cyl = set()
for tuple in interesting:
    G = Sn.subgroup([tuple.g(0), tuple.g(1), tuple.g(2), tuple.g(3)])
    cyclic_one_cyl.add(tuple) if G.is_cyclic() else non_cyclic_one_cyl.add(tuple)

print(f'Number of degree {n} one-cylinder PTS that are: \n cyclic: {len(cyclic_one_cyl)}')

non_cyclic_one_cylinder_non_identity = set()
identity = set()
for tuple in non_cyclic_one_cyl:
    if check_for_identity(tuple,n):
        identity.add(tuple) 
    else:
        non_cyclic_one_cylinder_non_identity.add(tuple)
        
print(f' non-cyclic with identity perm: {len(identity)}')
print(f' non-cyclic without identity perm: {len(non_cyclic_one_cylinder_non_identity)}\n')


Number of degree 5 one-cylinder PTS that are: 
 cyclic: 480
 non-cyclic with identity perm: 1920
 non-cyclic without identity perm: 66720



In [ ]:
cycle_types_set = set()
for tuple in non_cyclic_one_cylinder_non_identity:
    cycle_types_set.add(tuple.profile())

print(f'Number of cycle types: {len(cycle_types_set)}')

dedup_cycle_types_set = set()
for t in cycle_types_set:
    duplicate = False
    for perm in SymmetricGroup(4):
        permuted = (t[perm(1)-1], t[perm(2)-1], t[perm(3)-1], t[perm(4)-1])
        if permuted in dedup_cycle_types_set:
            duplicate = True
        
    if not duplicate:
         dedup_cycle_types_set.add(t)
print(f'Number of cycle types up to permutation: {len(dedup_cycle_types_set)}') # note: using isom type above produces same list
for t in dedup_cycle_types_set:
     print(t)
    

Number of cycle types: 136
Number of cycle types up to permutation: 26
([2, 2, 1], [2, 2, 1], [5], [2, 2, 1])
([4, 1], [2, 1, 1, 1], [2, 1, 1, 1], [4, 1])
([3, 2], [3, 2], [4, 1], [4, 1])
([4, 1], [2, 1, 1, 1], [4, 1], [4, 1])
([3, 1, 1], [3, 1, 1], [5], [5])
([5], [5], [5], [3, 1, 1])
([2, 2, 1], [3, 1, 1], [2, 2, 1], [5])
([4, 1], [2, 1, 1, 1], [3, 2], [3, 2])
([5], [5], [5], [2, 2, 1])
([4, 1], [4, 1], [4, 1], [3, 2])
([3, 2], [4, 1], [4, 1], [2, 1, 1, 1])
([3, 1, 1], [3, 1, 1], [5], [3, 1, 1])
([2, 2, 1], [3, 1, 1], [3, 1, 1], [3, 1, 1])
([2, 2, 1], [5], [2, 2, 1], [5])
([5], [3, 1, 1], [2, 2, 1], [5])
([3, 1, 1], [2, 2, 1], [3, 1, 1], [2, 2, 1])
([4, 1], [2, 1, 1, 1], [2, 1, 1, 1], [3, 2])
([2, 2, 1], [2, 2, 1], [2, 2, 1], [2, 2, 1])
([3, 2], [4, 1], [3, 2], [3, 2])
([2, 2, 1], [2, 2, 1], [3, 1, 1], [2, 2, 1])
([4, 1], [4, 1], [4, 1], [4, 1])
([3, 1, 1], [3, 1, 1], [3, 1, 1], [3, 1, 1])
([3, 1, 1], [2, 2, 1], [5], [3, 1, 1])
([5], [5], [5], [5])
([3, 2], [2, 1, 1, 1], [3, 2], [3, 

In [ ]:
# collect examples for each cycle type that appears
examples = {}
for cycle_type in dedup_cycle_types_set:
    for tuple in non_cyclic_one_cylinder_non_identity:
        if tuple.profile() == cycle_type:
            examples.update({cycle_type: tuple})
            break 

In [275]:
for (cycle_type, tuple) in examples.items():
    print_PTS(tuple,n,verbose=True)

PTS defined by:
 a: (1,3)(4,5)
 b: (1,2)(3,5)
 c: (1,5,3,4,2)
 d: (1,3)(2,5)

cycle type: ([2, 2, 1], [2, 2, 1], [5], [2, 2, 1])
is cyclic: False
ab = (1,5,4,3,2)
is 1-cylinder: True
Braid group orbit:
((2,3)(4,5),  (1,2)(3,4),  (1,2,4,3,5),  (1,4)(2,3))
a'b'=(1,2,4,5,3)
((2,3)(4,5),  (1,2)(3,4),  (1,4,5,3,2),  (2,5)(3,4))
a'b'=(1,2,4,5,3)
((2,3)(4,5),  (1,2,4,3,5),  (2,4)(3,5),  (1,4)(2,3))
a'b'=(1,2,5,3,4)
((2,3)(4,5),  (1,2)(3,4),  (1,4)(2,3),  (1,2,5,4,3))
a'b'=(1,2,4,5,3)
((2,3)(4,5),  (1,2)(3,4),  (1,2,5,4,3),  (1,5)(2,3))
a'b'=(1,2,4,5,3)


PTS defined by:
 a: (2,4,5,3)
 b: (1,5)
 c: (1,5)
 d: (2,3,5,4)

cycle type: ([4, 1], [2, 1, 1, 1], [2, 1, 1, 1], [4, 1])
is cyclic: False
ab = (1,5,3,2,4)
is 1-cylinder: True
Braid group orbit:
((2,3,4,5),  (1,2),  (1,2),  (2,5,4,3))
a'b'=(1,2,3,4,5)
((4,5),  (1,2,3,4),  (4,5),  (1,5,3,2))
a'b'=(1,2,3,4,5)
((2,3,4,5),  (1,2),  (2,5,4,3),  (1,5))
a'b'=(1,2,3,4,5)


PTS defined by:
 a: (1,4,2)(3,5)
 b: (1,5,2)(3,4)
 c: (1,3,4,5)
 d: (1,2,3,4)
